<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [26]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [27]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
data = df1.merge(df2, on='id',how='left')
#df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
#df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
#df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)
#df4_q1 = pd.DataFrame(df3.tfidf_fea_q1.values.tolist(), index= df3.index)
#df4_q2 = pd.DataFrame(df3.tfidf_fea_q2.values.tolist(), index= df3.index)

In [28]:
data.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,...,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,...,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,...,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,...,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,...,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [29]:
data = data.sample(50000)

In [30]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12


In [31]:
y_true = data['is_duplicate']

In [32]:
X_train,X_test, y_train, y_test = train_test_split(data, y_true, stratify=y_true, test_size=0.3)

In [33]:
print("Shape of Train dataset", X_train.shape)
print("Shape of Test dataset", X_test.shape)

Shape of Train dataset (35000, 28)
Shape of Test dataset (15000, 28)


In [34]:
# avoid decoding problems
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [35]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
df_train = df.loc[X_train.index]
df_test = df.loc[X_test.index]


questions = list(df_train['question1']) + list(df_train['question2'])
questions_test = list(df_test['question1']) + list(df_test['question2'])
tfidf = TfidfVectorizer(lowercase=False)
tfidf_questions = tfidf.fit_transform(questions)
# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
tfidf_test = tfidf.transform(questions_test)
word2tfidf_test = dict(zip(tfidf.get_feature_names(),tfidf.idf_))
print("Shape of Tfidf vec:", tfidf_questions.shape)
print("Few of the feature names: ", tfidf.get_feature_names()[:20])

Shape of Tfidf vec: (70000, 33764)
Few of the feature names:  ['00', '000', '0019', '00am', '00s', '01', '014', '018v4', '02', '03', '04', '04009', '04LTS', '05', '06', '062', '0706', '08', '09', '099']


- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [37]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_md')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df_train['question1'])):
    doc1 = nlp(qu1) 
    # 96 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 300])
    
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
           
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df_train['q1_feats_m'] = list(vecs1)


100%|███████████████████████████████████████████████████████████████████████████| 35000/35000 [04:14<00:00, 137.48it/s]


In [38]:
vecs2 = []
for qu2 in tqdm(list(df_train['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), 300])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df_train['q2_feats_m'] = list(vecs2)

100%|███████████████████████████████████████████████████████████████████████████| 35000/35000 [04:15<00:00, 137.10it/s]


In [39]:

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df_test['question1'])):
    doc1 = nlp(qu1) 
    # 96 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 300])
    
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
           
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df_test['q1_feats_m'] = list(vecs1)


100%|███████████████████████████████████████████████████████████████████████████| 15000/15000 [01:48<00:00, 137.91it/s]


In [40]:
vecs2 = []
for qu2 in tqdm(list(df_test['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), 300])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df_test['q2_feats_m'] = list(vecs2)

100%|███████████████████████████████████████████████████████████████████████████| 15000/15000 [01:48<00:00, 138.42it/s]


In [41]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats_m,q2_feats_m
73380,73380,125919,125920,What do you mean by viricide?,What percentage of educated Indians in their 2...,0,"[-3.8972494080662727, 1.0405236631631851, -6.8...","[-26.99588058143854, 17.520919024944305, 1.322..."
95453,95453,159189,159190,What is the evolutionary explanation of homose...,What is the Darwinian explanation for homosexu...,1,"[-3.8326794877648354, 1.912333607673645, -4.22...","[-4.960261531174183, 1.6103971004486084, -5.33..."
18532,18532,35104,3480,What are all the best places to visit in goa?,"We are Planning to visit Goa for three days,wh...",1,"[-3.9776771143078804, 3.7677771896123886, -2.2...","[3.9453700184822083, 6.001156955957413, -9.320..."
291197,291197,412582,412583,If f(x) = (x^2+x+1) / (2x+7) and g(x) = (x^2+x...,What is 2x+7?,0,"[-2.7263535261154175, 5.132687211036682, -4.46...","[-0.2771619036793709, 2.2657933235168457, -0.4..."
325882,325882,75909,452136,Gary Johnson is the Libertarian candidate for ...,"Which candidate will Gary Johnson hurt more, C...",1,"[-24.067868798971176, 28.315082848072052, 4.48...","[-19.847059696912766, 12.705795735120773, -5.3..."


In [42]:
# Questions 1 tfidf weighted word2vec
df_test.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats_m,q2_feats_m
343232,343232,471320,127014,How many Hindu nations are in the world?,"What are First World, Second World and Third W...",0,"[-6.976658832281828, 0.4694499373435974, 2.846...","[1.477362371981144, 11.920952379703522, 1.9978..."
287712,287712,198406,179012,What is the least painless way to kill yourself?,"What would be a sure shot, painless way to kil...",1,"[-11.563796944916248, 6.492379132658243, -12.9...","[-10.543945260345936, 9.215830109082162, -16.7..."
31718,31718,58458,58459,Which is the type of a new and stylish busines...,I am new to business marketing. I want to star...,0,"[7.0781506560742855, 11.295192271471024, -5.67...","[-29.03106477484107, 45.0119488760829, -15.061..."
19452,19452,36756,36757,How much marks in need in mains to get ECE in ...,How much marks in need in mains to get ECE in ...,0,"[-9.527374925557524, 18.180076494812965, 10.95...","[-9.643484028521925, 12.730724230408669, 7.548..."
212578,212578,104629,24793,What is your New Year Resolution?,What's your New Year 2017 resolution?,1,"[-1.2461856678128242, 10.756372213363647, -7.5...","[3.6941706016659737, 10.240451097488403, -4.08..."


In [43]:
df3_train = df_train.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df_train_q1 = pd.DataFrame(df3_train.q1_feats_m.values.tolist(), index= df3_train.index)
df_train_q2 = pd.DataFrame(df3_train.q2_feats_m.values.tolist(), index= df3_train.index)

###################################################################################

df3_test = df_test.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df_test_q1 = pd.DataFrame(df3_test.q1_feats_m.values.tolist(), index= df3_test.index)
df_test_q2 = pd.DataFrame(df3_test.q2_feats_m.values.tolist(), index= df3_test.index)

In [44]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df_train_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df_train_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df_train_q1.shape[1]+df_train_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 300
Number of features in question2 w2v  dataframe : 300
Number of features in final dataframe  : 629


In [45]:
# storing the final features to csv file
if not os.path.isfile('final_features_train.csv'):
    df_train_q1['id']=X_train['id']
    df_train_q2['id']=X_train['id']
    
    df2  = df_train_q1.merge(df_train_q2, on='id',how='left')
    X_train  = X_train.merge(df2, on='id',how='left')
    X_train.to_csv('final_features_train.csv')

if not os.path.isfile('final_features_test.csv'):
    df_test_q1['id']=X_test['id']
    df_test_q2['id']=X_test['id']
    
    df2  = df_test_q1.merge(df_test_q2, on='id',how='left')
    X_test  = X_test.merge(df2, on='id',how='left')
    X_test.to_csv('final_features_test.csv')

In [47]:
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')